# Cliopatria viewer

1. To get started, download a copy of the Cliopatria dataset from here: `[INSERT LINK]`
2. `(OPTIONAL)` You can also download the GADM modern borders dataset from here `[INSERT LINK]`
3. Move the downloaded shape dataset(s) to an appropriate location on your machine and pass in the paths in the code cell below and run
4. Run the subsequent cells of the notebook


In [ ]:
cliopatria_path = "path/to/cliopatria_dir/"
gadm_path = "path/to/gadm_410.gpkg"